# Evaluate Base Model Endpoints using Azure AI Evaluation APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [ ]:
%pip install azure-ai-evaluation
%pip install promptflow-azure
%pip install promptflow-tracing
%pip install promptflow-evals

### Parameters and imports

In [ ]:
from pprint import pprint

import pandas as pd
import random
import os
from dotenv import load_dotenv
load_dotenv("../.credentials.env")

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [4]:
env_var = { 
    "gpt-35-turbo": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT35_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT35_API_KEY"),
    },
    "gpt-4": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT4_API_KEY"),
    },
    "gpt-4o": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4o_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT4o_API_KEY"),
    },
   "gpt-4o-mini" : { 
        "endpoint" : os.environ.get("AZURE_OPENAI_GPT4o-mini_ENDPOINT"), 
        "key" : os.environ.get("AZURE_OPENAI_GPT4o-mini_API_KEY"), 
    },    
}


Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [5]:
# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

## Model Endpoints
The following code demonstrates how to call various model endpoints, and is configured based on `env_var` set above. For any model in `env_var`, if you do not have that model deployed in your AI project, please comment it out. If you have a model that you would like to test that does not correspond with one of the types seen below, please include that type in the `__call__` function and create a helper function to call the model's endpoint via REST. 

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [ ]:
df = pd.read_json("../datasets/ai_data.jsonl", lines=True)
print(df.head())

## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [9]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [ ]:
with open("target_api/target_api.py") as fin:
    print(fin.read())

In [11]:
from target_api.target_api import ModelEndpoints

In [ ]:
import pathlib
import time
from target_api.target_api import ModelEndpoints
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)

relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)


models = ["gpt-35-turbo","gpt-4","gpt-4o","gpt-4o-mini"]

path = str(pathlib.Path(pathlib.Path.cwd())) + "/ai_data.jsonl"

for model in models:
    print(" Evaluating AI-assisted metrics - ", model)
    print("-----------------------------------")
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        azure_ai_project=azure_ai_project, 
        evaluation_name="Eval_" + model.title() + "_Run-" + str(randomNum),
        data=path,
        target=ModelEndpoints(env_var, model),
        
        evaluators={
            "relevance": relevance_evaluator,
            "groundedness": groundedness_evaluator,
            "coherence": coherence_evaluator,
            "fluency": fluency_evaluator,
            "similarity": similarity_evaluator,

        },
        evaluator_config={

            "relevance": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
                },

            "groundedness": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "coherence": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "fluency": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "similarity": {
                "column_mapping": {
                    "ground_truth": "${data.ground_truth}",
                    "context": "${data.context}", 
                    "query": "${data.query}",
                    "response": "${target.response}"}
            },
        },
    )
    #time.sleep(60) ## To avoid rate limiting throttling


View the results

In [ ]:
pd.DataFrame(results["rows"])